In [ ]:
bronze_london_authorities_table = '`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-bronze-001`.london_local_authorities'
silver_london_authorities_table = '`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-silver-001`.london_local_authorities'

bronze_table = '`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-bronze-001`.environmental_data'
silver_table = '`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-silver-001`.environmental_data'


In [ ]:
# silver london authority data
silver_london_authorities_table = '`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-silver-001`.london_local_authorities'
bronze_london_local_authority_data = spark.table('`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-bronze-001`.london_local_authorities')
bronze_london_local_authority_data.write.mode('overwrite').saveAsTable(silver_london_authorities_table)

In [0]:
from pyspark.sql.functions import col, to_timestamp
from delta.tables import DeltaTable

bronze_environmental_data = spark.table(bronze_table)
# show top 5 of bronze table
bronze_environmental_data.limit(5).display()

silver_environmental_data = (
    bronze_environmental_data
    .withColumnRenamed('Site', 'site')
    .withColumnRenamed('Species', 'species')
    .withColumnRenamed('ReadingDateTime', 'reading_datetime')
    .withColumn('reading_datetime', to_timestamp(col('reading_datetime'), 'dd/MM/yyyy HH:mm'))
    .withColumnRenamed('Value', 'value')
    .withColumn('value', col('value').cast('float'))
    .withColumnRenamed('Units', 'units')
    .withColumnRenamed('Provisional_or_Ratified', 'data_status')
    .filter(col("value").isNotNull())
)

if not spark.catalog.tableExists(silver_table):
    (silver_environmental_data
        .limit(0)                     # no rows, only schema
        .write
        .format("delta")
        .saveAsTable(silver_table))


# save silver data
silver_data = DeltaTable.forName(spark, silver_table)
(
    silver_data.alias("t")
    .merge(
        silver_environmental_data.alias("s"),
        """
        t.site = s.site
        AND t.species = s.species
        AND t.reading_datetime = s.reading_datetime
        """
    )
    .whenNotMatchedInsertAll()
    .execute()
)

silver_data.toDF().show(5)

